In [9]:
import os
%pwd

'g:\\PROJECT\\Text-Summarizer\\research'

In [10]:
os.chdir("../")

In [11]:
pwd

'g:\\PROJECT\\Text-Summarizer'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [13]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories




In [14]:
class ConfigurationManager:
    def __init__(self,
                config_path=CONFIG_FILE_PATH,
                params_path=PARAMS_FILE_PATH):
        self.config= read_yaml(config_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config
        

In [23]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk
from datasets import DatasetDict

## DATA TRNASFORMATION COMP0ONENT

In [24]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_name)


    def cvrt_examples_to_features(self,ex_batch):
        
        input_encodings=self.tokenizer(ex_batch['dialogue'], max_length=511, truncation=True)
        with self.tokenizer.as_target_tokenizer():
         target_encodings=self.tokenizer(ex_batch['summary'], max_length=180, truncation=True)

        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask' :input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']

            }


    def convert(self):
        ds=load_from_disk(self.config.data_path)
        small_train=ds['train'].select(range(2000))
        small_test=ds['test'].select(range(300))
        small_valid=ds['validation'].select(range(300))
        ds1=DatasetDict({
           'train': small_train,
           'validation': small_valid
        })
        ds1=ds.map(self.cvrt_examples_to_features, batched=True)
        ds1.save_to_disk(os.path.join(self.config.root_dir, "transformed_dataset"))


In [20]:
from datasets import load_from_disk

ds = load_from_disk("artifacts/data_transformation/transformed_dataset/")
ds


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [26]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2025-12-11 00:23:19,170: INFO: common: yaml file config\config.yaml loaded successfully:]
[2025-12-11 00:23:19,174: INFO: common: yaml file params.yaml loaded successfully:]
[2025-12-11 00:23:19,176: INFO: common: created directory atartifacts:]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 44729.04 examples/s]


FileNotFoundError: Directory artifacts/data_transformation/transformed_dataset not found